# Création d'un dictionnaire allemand pour Iramuteq

Pour l'instant, il [n'existe que](http://www.iramuteq.org/dictionnaires) le dictionnaire par défaut français et les trois langues suivantes:
- Néerlandais
- Norvégien
- Grec

Le but étant d'en créer un allemand pour pouvoir utiliser les textes des journaux suisses-allemands

## Etape 1: Récupérer une liste des mots allemands

Suivant les instructions de ce lien: http://www.danielnaber.de/morphologie/

1. Télécharger la version offline de [Language Tool](https://languagetool.org/)
2. Télécharger les données du [Morphologie Lexikon](https://github.com/languagetool-org/german-pos-dict/releases)
3. Extraire les données et "Language Tool"
4. Adapter le chemin vers le jar de "Lanugage Tool" dans le fichier `export.sh` des données
5. Run `export.sh` pour générer `dictionnary.dump`

C'est un fichier avec des séparations de tab `\t` avec les déclinaisons des mots et le mot principal sur chaque ligne (et des détails grammaticaux)

In [69]:
PATH_TO_DATA = "german-pos-dict-1.1/dictionary.dump"
file = open(PATH_TO_DATA, "r")
words = file.readlines()
file.close()

Exemple:

In [70]:
words[2000].split("\t")

['Abbruchreiferen', 'Abbruchreifere', 'SUB:NOM:PLU:FEM:ADJ\n']

Catégories des mots allemands -> à remapper pour Iramuteq
```
'SUB': 1085004,
'ADJ': 908956,
'EIG': 15750,
'VER': 281402,
'ADV': 484,
'KON': 81,
'ABK': 58,
'PRP': 119,
'PA1': 988933,
'ZUS': 152,
'PA2': 965432,
'INJ': 18,
'ZAL': 32,
'PRO': 754,
'ART': 36,
'NEG': 6
```
   
Iramuteq utilise différentes catégorisations qui ne sont pas uniformisées dans les dicitonnaires des autres langues dispo donc je me base sur celui en français résupérés des fichier d'installation du programme.

In [67]:
# Chercher type de mots utilisés dans les dictionnaires grec, néerlandais, etc...

file2 = open("iramuteq/lexique_fr.txt", "r")
words2 = file2.readlines()
file2.close()

categories = []

for word in words2:
    splits = word.split("\t")
    categories.append(splits[2].rstrip())
    
Counter(categories)

Counter({'aux': 85,
         'pre': 80,
         'nom': 43137,
         'adv': 1615,
         'ono': 233,
         'ver': 60158,
         'adj': 19636,
         'adv_sup': 89,
         'con': 31,
         'nom_sup': 75,
         'art_def': 9,
         'adj_ind': 30,
         'pro_rel': 16,
         'pro_ind': 24,
         'adj_sup': 36,
         'pro_dem': 17,
         'adj_num': 106,
         'adj_dem': 3,
         'pro_per': 46,
         'ver_sup': 258,
         'art_ind': 4,
         'adj_pos': 14,
         'pro_pos': 15,
         'adj_int': 4})

In [68]:
# Création du dictionnaire:

dictDE_FR = {
    "SUB": "nom",
    "ADJ": "adj",
    "EIG": "nom", # Pays
    "VER": "ver",
    "ADV": "adv",
    "KON": "con",
    "ABK": "adv", # Abréviation
    "PRP": "pre", # prépositions
    "PA1": "adj", # adjectif déclinés
    "ZUS": "adv",
    "PA2": "adj",
    "INJ": "adv",
    "ZAL": "num",
    "PRO": "pro_per",
    "ART": "art_ind",
    "NEG": "pro_per"
}

In [86]:
# Remplacer les catégorisations allemandes par celles d'Iramuteq
# enlever duplicatas

data = []
set_ = {"_"}

for word in words:
    splits = word.split("\t")
    if splits[0] == "-" or (splits[0] in set_):
        continue
    else:
        set_.add(splits[0])
    category = splits[2][:3]
    splits[2] = dictDE_FR.get(category)
    data.append(splits)

In [99]:
data[2457]

['Abkommandierungen', 'Abkommandierung', 'nom']

In [100]:
# Exporter le tout dans un fichier txt
PATH_TO_RESULT = "iramuteq/lexique_de.txt"
result = open(PATH_TO_RESULT, "w+")

for word in data:
    result.write(word[0] + "\t" + word[1] + "\t" + word[2] + "\n")

result.close()